# Proyecto 1: Brecha Digital
### Preparación de datos

#### Basado en el reporte ["La Brecha Digital en California"](https://www.ppic.org/wp-content/uploads/jtf-californias-digital-divide-in-spanish.pdf)

## Pregunta(s) de Investigación:
1. ¿Qué porcentaje de hogares en el estado X tiene acceso a internet de alta velocidad?
2. ¿Varía este número según los grupos demográficos? (en este caso raza/etnia.

## Meta:
* Explorar los archivos de datos (`acsdata.data.gz`) y crear un _conjunto de datos analítico_ (un archivo derivado enfocado específicamente en análisis en mano).

## Contexto:
Obtenimos datos de la encuesta American Community Survey (ACS) de [IPUMS](https://usa.ipums.org/usa/). <br>
Contiene características demográficas:
  - edad
  - sexo
  - raza/etnia

e indicadores geográficos:
  - estado
  - condado

***

#### Paso 1: Prepara tu entorno de trabajo.

**Import**a las bibliotecas necesarias y crea objetos `Path` (ruta de archivo). Esto grarantiza reproducibilidad en distintos sistemas operativos (Windows utiliza `\` en lugar de `/` para separar los nombres de archivos.

Necesitamos: 
1. `pandas` para trabajar con los datos.
2. `pathlib`, y más específicamente su objeto `Path`, para trabajar con rutas de archivos. Esto asegura que nuestro código funcione en Windows (que utiliza `\` en sus rutas) y MacOS/Linux (los cuales utilizan `/`).
3. `datetime` - tip: Existen sistemas de control de versiones para datos pero etiquetar tus archivos de datos (cuando no son masivos) no es un mal primer paso si estás comenzando.

In [1]:
# Prepara tu entorno de trabajo
import _____ as pd
from _____ import Path
from datetime import datetime as dt
today = __.today().strftime("%d-%b-%y")

print(hoy)

27-Apr-19


_nota: Aunque estés utilizando Windows puedes escribir_ `/` _en tus rutas de archivo._

In [2]:
# Directorio de datos y rutas
RUTA_DATOS_EN_BRUTO = ____("../datos/brutos/")
RUTA_DATOS_________ = ____("../datos/interinos/")
RUTA_______________ = ____("../datos/procesados/")
___________________ = ____("../datos/finales/")

**NOTA:** He incluido un _script_ de `python` llamado `herramientas.py` con la función `arbol` la cual muestra el árbol de directorios. La obtuvé de el [tutorial de RealPython's sobre el módulo `pathlib`](https://realpython.com/python-pathlib/)).

    de nuestro script herramientas importa arbol para utilizarla

In [ ]:
arbol(________)

***

#### Paso 2: Carga y explora los datos

Con `pandas` _leer_ archivos de datos es tan fácil como escribir `.read_csv(RUTA_AL_ARCHIVO_CSV)` y esto es suficiente la mayoría del tiempo. La función `read_csv()` de `Pandas` es tan poderosa que incluso lee archivos comprimidos sin tener que especificar algún otro parametro. Prueba lo siguiente:

```python
datos = pd.read_csv(RUTA_DATOS_EN_BRUTO / 'acs_data.csv.gz')
datos.head()
```
_Asegurate de cambiar_ `RUTA_DE_DATOS_EN_BRUTO` _a lo que seaque hayas llamado tu variable a la que asignaste el objeto `Path` con la ruta a tus datos en bruto._

***
IPUMS ofrece algunos formatos de datos que pueden ser aún más útiles [[docs]](https://usa.ipums.org/usa-action/faq#ques12):
> Además del archivo de datos ASCII, el sistema crea un archivo de sintaxis de paquetes estadísticos para acompañar cada extracto. El archivo de sintaxis está diseñado para leer los datos ASCII mientras se aplican las variables de valor y las etiquetas apropiadas. SPSS, SAS y Stata son compatibles. Debe descargar el archivo de sintaxis con el extracto o no podrá leer los datos. El archivo de sintaxis requiere una edición menor para identificar la ubicación del archivo de datos en su computadora local.

En este caso, usaremos un archivo **Stata** (`.dta`). La razón principal es que los archivos `.dta` pueden almacenar *etiquetas de valor* que` pandas` luego puede leer y convertir columnas en columnas `Categóricas` en nuestro marco de datos de pandas. Esto 1) ahorra memoria, y 2) es una buena práctica porque ciertas ciencias sociales en serio, en serio _en serio_, y en serio _en serio_ ***en serio*** aman a Stata, por lo que sus conjuntos de datos interesantes son probablemente archivos `.dta`.

Sin embargo, `pandas` no puede leer` .dta` comprimido directamente como lo hace con los archivos `.csv`. Afortunadamente, IPUMS utiliza el formato comprimido *gzip* y `python` incluye un módulo` gzip` en su biblioteca estándar.

**Import**a gzip y prueba lo siguiente:
```python
with gzip.open(RUTA_DATOS_EN_BRUTO / 'acs_data.dta.gz') as archivo:
    datos = pd.read_stata(archivo)
```

y muestra las primeras cinco filas de tu `datos` _DataFrame_.

In [ ]:
# importa gzip y carga tus datos




In [ ]:
# muestra las primeras 5 filas


***

#### Paso 3: Familiarizate con el conjunto de datos.
Ya hemos visto `.head()` - el método de `pandas` que mostrará las primeras 5 filas de tu DataFrame. Esto te da una idea de cómo se ven sus datos. Sin embargo, hay mucho más `.info()` que puede salir de su marco de datos. También puede simplemente pedirle al DataFrame si los `.describe()`...

In [ ]:
# averigua más info de tu DataFrame
data.____()

In [ ]:
# describe tus datos
data.____()

Comprueba la 'forma' de tus datos con su atributo `.shape`. Note la falta de paréntesis.

In [ ]:
data._____

***

#### Paso 4: Recorta tus datos
En este momento estás trabajando con tu **archivo maestro**: un conjunto de datos que contiene todo lo que _podría_ necesita para su análisis. Realmente no deseas modificar este conjunto de datos porque podría estar usándolo para otros análisis. Por ejemplo, vamos a analizar el acceso a Internet de alta velocidad en un estado de tu elección, pero la próxima semana es posible que desees realizar el mismo análisis en otro estado o tal vez solo en un condado específico. Para asegurarse de que puedas **reutilizar** tus datos y tu código más adelante, creamos un _archivo de análisis_, un conjunto de datos que contiene solo los datos necesarios para **este** análisis específico a mano.

Primero, solo estamos interesados en encontrar la _ "Brecha Digital" de un estado en este momento. El **archivo maestro** contiene datos de los 50 estados y el distrito de Columbia.

Lo que quieres hacer es encontrar todas las filas donde el `statefip` coincide con el nombre de tu estado. Esto se llama indexación booleana.

Prueba lo siguiente:
```python
data['statefip'] == 'ohio'
```
_Nota: Tu puedes cambiar 'ohio' a cualquiera de los 50 estados o 'district of columbia' para DC._

In [ ]:
# Prueba indexación booleanda
___['______'] == '_______'

Esto devolverá un `pandas.Series` de booleanos (Trues y Falses) que luego puede usar para filtrar las filas innecesarias.

Es una buena práctica guardar esta _Serie_ como una variable al principio de tu código (si la conoces de antemano) o justo antes de usarla en caso de que use estas condiciones en más de un lugar. Esto te ahorrará tiempo si decide cambiar el valor que está comparando, `'ohio'` para`' california'` por ejemplo.

```python
mascara_estado = (datos['statefip'] == 'ohio')
datos[mascara_estado].head()
```

In [ ]:
# hazlo tu
mascara_estado = (________________________ == _______)
datos[mascara_estado].____()

Guardemoslo en una variable con un nombre más útil:

```python
datos_estatales = datos[mascara_estado].copy()
```

Tienes que utilizar `.copy()` para crear copias _reales_ de los datos. Si ejecutas:
```python
datos_estatales = datos[mascara_estado]
```
`datos_estatales` sería lo que la documentación de `pandas` se refiere como una _vista_ de DataFrame `datos`. Esto puede tener consecuencias más adelante si modificas estos DataFrames. Muchas veces recibirás solo una advertencia y tu código se ejecutará tal como lo esperabas - pero, ¿para qué tomar riesgos?

In [ ]:
# asigna tu nuevo DataFrame a la variable datos_estatales
datos_estatales = __________.copy()

Ahora, veamos que `.columns` tenemos en nuestro DataFrame. Puedes acceder esta información de la misma manera que encontramos la _"forma"_ hace un rato.

In [ ]:
datos_estatales._____

¿Hay columnas de las cuales **tienes seguridad** que no vas a necesitar? <br> Si no estás 90% seguro de que no vas a necesitar una columna, no te deshagas de ella.

Eliminar columnas es tan fácil como utilizar `.drop()` en tu DataFrame.

```python
datos_estatales.drop(columns = ['lista', 'de', 'columnas', 'pa', 'eliminar'])
```

Si hay columnas que tu _crees_ que no vas a necesitar pero no tienes mucha seguridad de que ese es el caso, deberías explorarlas.

Las columnas de los DataFrames de `pandas` son `pandas.Series` y tienen métodos y atributos como los DataFrames.

Exploremos la columna `gq` que representa los valores de la variable `Group Quarters`. La [documentación](https://usa.ipums.org/usa-action/variables/GQ#description_section) de IPUMS define _Group Quarters_ así:

>Group quarters son en su mayoría instituciones y otras viviendas para grupos, por ejemplo, barracas militares y dormitorios.

Exploremos que valores _unicos_ (`.unique()`) tiene la Serie `datos_estatales['gq']`...

Tambien podemos ver las cuentas totales de los sus valores (`.value_counts()`) lo que nos daría una mejor idea de que tan útil esta columna podría ser.<br>
Por ejemplo, si una columna tiene 2 valores pero 99% de sus observaciones tiene un valor y el resto tiene otro - en este caso, podrías deshacerte de esa columna ya que no agregaría mucho valor a tu análisis.

<i><span style="font-size:12px"> Hay algunas variables donde el 100% de sus filas tienen el mismo valor \*tos\* \*tos\* </i><span style="font-family:monospace">datos_estatales['year'] </span>

De la [documentación](https://usa.ipums.org/usa-action/variables/GQ#comparability_section) de IPUMS:
>Hay tres definiciones un poco diferentes de _group quarters_ en IPUMS. Para el periodo 1940-1970 (excluyendo el conjunto de datos 100% de 1940), _group quarters_ eran unidades de vivienda con 5 o más individuos sin relación a la cabeza de la vivienda. Antes de 1940 y en 1980-1990, unidades con 10 o más individuos sin relación a la cabeza de la vivienda eran considerados _group quarters_. **En el censo del 2000, 2010, la ACS y la PRCS, ningún límite fue aplicado; para que un hogar fuera considerado _group quarter_, tenía que estar en una lista de _group quarters_ que el Censo mantiene continuamente. En años pasados, una lista similar fue utilizada, con la regla sobre personas-sin-relación immpuesta como seguridad.** 

Por esta razón, y por el hecho de que la gran mayoría de nuestras observaciones caen bajo la definición de 1970 y 1990, estas son a las que nos apegaremos para nuestro análisis.

Creemos otra _mascara_ para filtrar todos los hogares que no caen bajo nuestra definición

Para evaluar multiples condiciones utilizamos los operadores `&` y `|` ("**Y**" y "__O__", respectivamente).

In [ ]:
mascara_hogar = ( PRIMERA CONDICION ) | ( SEGUNDA CONDICION )

**nota**: Otro valor agregado al utilizar variables de tipo _categórico_ es que ***sí*** son valores ordenados puedes utilizar los operadores `<` y `>` para evaluar condiciones.

```python
mascara_hogar = (datos_estatales['gq'] <= 'additional households under 1990 definition')
```

<summary><i>nota</i>:
    <details value="nota">
        <i>Porque estamos sobreescribiendo</i> <span style="font-family:monospace">datos_estatales</span> <i>no necesitas utilizar</i> <span style="font-family:monospace">.copy()</span> <i>pero no nos afecta utilizarlo. Además, si eres principiante en</i> <span style="font-family:monospace">pandas</span> <i>es buena practica para cuando en serio necesites utilizar</i> <span style="font-family:monospace">.copy()</span>.
    </details>
</summary>

In [ ]:
datos_estatales = datos_estatales[mascara_hogar].______()

Para este momento ya estas muy cerca de crear un `datos_analisis` DataFrame. Hasta ahora:
1. Mantuviste los datos de un solo estado y te deshiciste del resto.
2. Mantuviste los datos de aquellos _hogares_ en los que tenemos interés y te deshiciste del resto.

Nuestra pregunta de investigación 1 es: _"¿Qué porcentage de hogares en el estado X tiene acceso a internet de alta velocidad?"_

Matemáticamente, 
$$ \frac{hogares\ con\ internet\ de\ alta\ velocidad}{hogares\ en\ el\ estado}$$

Tu conjunto de datos `datos_estatales` ahora contiene todo lo que necesitas para encontrar la respuesta.

***

#### Paso 5: Guarda tus datos
Ahora que haz recordado tu **archivo maestro** en un conjunto de datos enfocado para tu análisis, deberías guardarlo.

Hemos estado trabajando con un archivo `.dta` y sale mejor mantenerlo así.

Prueba lo siguiente:
```python
datos_estatales.to_stata(RUTA_DATOS_INTERINOS / f'datos_estatales-{hoy}.dta', write_index = False)
```

Un par de cosas:
1. Estamos utilizando `f-strings` para etiquetar nuestro archivo de datos con la fecha de hoy.
2. Estamos apagando la bandera `write_index` para que al grabar nuestro archivo no se le agregue una columna "index" (índice). Es nuestro conjunto de datos el índice no tiene valor. En otros análisis puede que tengas un índice que tenga algún valor o significado - en esos casos no vas a querer _apagar_ esta bandera.

***

#### Paso 6: Bonus
¿Qué tal si cambiaramos nuestra pregunta de investigación un poco? de <br> _"¿Qué porcentaje de hogares en el estado X tienen acceso a internet de alta velocidad?_" <br> a <br> _"¿Qué porcentaje de hogares **con niños y niñas de edad escolar** en el estado X tiene acceso a internet de alta velocidad?"_

Esto sería una estadística interecante para los creadores de políticas públicas, especialmente si encontramos diferencias entre grupos demográficos (pregunta de investigación 2).

El reto aquí es que nuestra **unidad de observación** en el archivo `datos_estatales` es una persona (ponderada) y que ahora queremos filtrar aquellos **hogares** sin niños o niñas de edad escolar. Esto puede sonar un poco complicado a primera vista pero solo requiere modificar nuestro flujo de trabajo un poco.

Necesitamos hacer unas cuantas cosas:
1. Definir a que nos referimos con _niños/niñas de edad escolar_.
2. Crear una _mascara_ para identificar todos los hogares donde esten estos niños o niñas.
3. Crear una lista sin duplicados de estas identificaciones de hogar (`'serial'`) 
4. Usar esta lista para eliminar todas las observaciones innecesarias.

#### Step 6.1: School-age children

Most people would agree school age (Elementary through High School) is 6 - 17 year olds. Some people are interested in K-12 (5 - 17 or 18). Some people wouldn't include 18 year olds. Whatever measure you choose you must be able to defend why you are choosing it. 

For this analysis, I'll suggest we use 5 - 18 year olds (K-12) but you can choose whatever age range you want. Maybe high-school kids 14-18? That'd be interesting, you probably need access to high-speed internet at home a lot more in high school than you do in kindergarden. 

In [ ]:
mask_children = (state_data['age'] >= ___) & (___________________ <= )

<summary> <i>What data type is</i> <span style='font-family:monospace'>state_data['age']</span> <i>again?</i> 
    <details> 
        Categorical. This means that you even though its values _look_ like numbers, they're actually _value labels_ aka strings.
    </details>
</summary>

Now that we have our _mask_, we can use it to create a list of households with children in them.

Earlier we applied a mask to a dataframe and saved it to another variable. Here, we'll go a step further and grab just a column of that _filtered out_ dataframe.

Try it yourself first.

*Hint: How did we grab and explore a single column of a dataframe earlier?*

In [ ]:
households_with_children = _________________________________________

In [ ]:
households_with_children.head()

How do you think we can `.drop_duplicates()`?

Once you have your unduplicated list of households with children all you have to do is to check if a `serial` value from our `state_data` dataset `.isin()` our `households_with_children` series.

Let's save that as our `working_data` dataset and save that to memory.

In [ ]:
working_data = _____________________

```python
working_data.to_stata(INTERIM_DATA_PATH / f'working_data-{today}.dta', write_index = False)
```